In [1]:
import os
import asyncio
import libsql_client
from dotenv import load_dotenv

# Try to load from .env file, but fall back to setting directly if not found
try:
    load_dotenv()
    print("✅ Loaded environment variables from .env file")
except:
    print("⚠️  No .env file found, using direct environment variable setting")
    
async def main():
    db_url = os.environ.get("TURSO_DATABASE_URL")
    auth_token = os.environ.get("TURSO_AUTH_TOKEN")
    
    if not db_url or not auth_token:
        print("Error: Please set TURSO_DATABASE_URL and TURSO_AUTH_TOKEN.")
        print("Either create a .env file with these variables or set them directly in the notebook.")
        return

    try:
        # --- THE FIX IS HERE ---
        # We replace "libsql" with "https" to force an HTTP connection
        # instead of the default WebSocket (wss), which can be blocked by firewalls.
        http_url = db_url.replace("libsql", "https", 1)
        
        async with libsql_client.create_client(url=http_url, auth_token=auth_token) as client:
            print("✅ Successfully connected to Turso DB over HTTPS!")
            
            # Test query
            result_set = await client.execute("SELECT sqlite_version();")
            print(f"SQLite version: {result_set.rows[0][0]}")

    except Exception as e:
        print("--- CONNECTION FAILED ---")
        print(f"An error occurred: {e}")
        print("Please double-check your Authentication Token.")

# Run the async function in the notebook
await main()

✅ Loaded environment variables from .env file
✅ Successfully connected to Turso DB over HTTPS!
SQLite version: 3.45.1


In [2]:
# Query article_topics table
async def query_article_topics():
    db_url = os.environ.get("TURSO_DATABASE_URL")
    auth_token = os.environ.get("TURSO_AUTH_TOKEN")
    
    if not db_url or not auth_token:
        print("Error: Please set TURSO_DATABASE_URL and TURSO_AUTH_TOKEN.")
        return

    try:
        # Use HTTPS connection
        http_url = db_url.replace("libsql", "https", 1)
        
        async with libsql_client.create_client(url=http_url, auth_token=auth_token) as client:
            print("🔍 Querying article_topics table...")
            
            # First, let's check if the table exists and get its structure
            result_set = await client.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='article_topics';")
            if not result_set.rows:
                print("❌ article_topics table not found!")
                return
            
            print("✅ article_topics table exists!")
            
            # Get table schema
            schema_result = await client.execute("PRAGMA table_info(article_topics);")
            print("\n📋 Table schema:")
            for row in schema_result.rows:
                print(f"  - {row[1]} ({row[2]})")
            
            # Get row count
            count_result = await client.execute("SELECT COUNT(*) FROM article_topics;")
            row_count = count_result.rows[0][0]
            print(f"\n📊 Total rows in article_topics: {row_count}")
            
            # Get sample data (first 5 rows)
            if row_count > 0:
                sample_result = await client.execute("SELECT * FROM article_topics LIMIT 5;")
                print(f"\n🔍 Sample data (first 5 rows):")
                for i, row in enumerate(sample_result.rows, 1):
                    print(f"  Row {i}: {row}")
            else:
                print("\n⚠️  No data found in article_topics table")

    except Exception as e:
        print("--- QUERY FAILED ---")
        print(f"An error occurred: {e}")

# Run the query
await query_article_topics()


🔍 Querying article_topics table...
✅ article_topics table exists!

📋 Table schema:
  - id (TEXT)
  - article_id (TEXT)
  - entity_text (TEXT)
  - entity_type (TEXT)
  - tfidf_score (REAL)
  - ner_confidence (REAL)
  - created_at (DATETIME)

📊 Total rows in article_topics: 1176

🔍 Sample data (first 5 rows):
  Row 1: ('topic_news_1760505480846_n4arng9hz9l_0_1760505482319', 'news_1760505480846_n4arng9hz9l', 'Hamas', 'ORG', 0.0, 0.95, '2025-10-15 05:18:02')
  Row 2: ('topic_news_1760505480846_n4arng9hz9l_1_1760505482319', 'news_1760505480846_n4arng9hz9l', 'Israel', 'LOCATION', 0.0, 0.95, '2025-10-15 05:18:02')
  Row 3: ('topic_news_1760505480846_n4arng9hz9l_2_1760505482319', 'news_1760505480846_n4arng9hz9l', 'Gaza', 'LOCATION', 0.0, 0.95, '2025-10-15 05:18:02')
  Row 4: ('topic_news_1760505481409_mtm4zm70tp_0_1760505482865', 'news_1760505481409_mtm4zm70tp', 'Bowen', 'PERSON', 0.0, 0.8, '2025-10-15 05:18:03')
  Row 5: ('topic_news_1760505481409_mtm4zm70tp_1_1760505482865', 'news_1760505481